# Scraping
Upphaflega fór skröpun fram í þessu skjali en ákvað að splitta því upp svo hægt væri að keyra alla gpt-þjálfunina á collab án þess að þurfa keyra allt draslið aftur

In [ ]:
%pip install datasets
%pip install tokenizers
%pip install transformers
%pip install seqeval
%pip install accelerate==0.24.1
%pip install wandb

> Hér gögnum við úr skugga um það að við séum með alla þá pakka sem þarf til að keyra notebookið

# GPT-2 þjálfun

## Formöttum gögn

In [1]:
import json
from sklearn.model_selection import train_test_split

with open('scraping/scraped_data.json') as f:
    data = json.load(f)

lyrics = list(data.values())
for i in range(len(lyrics)):
    lyrics[i] = "\n".join(lyrics[i])

def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        data += texts + "  "
    f.write(data)

train, test = train_test_split(lyrics,test_size=0.15)

build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 3332
Test dataset length: 588


> Skiptum gögnunum upp í train- og testdataset

## Gerum gögnin "þjálfanleg"

In [2]:
from transformers import TextDataset,DataCollatorForLanguageModeling, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("jonfd/gpt2-igc-is")

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.11/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (118490 > 512). Running this sequence through the model will result in indexing errors


## Stillum þjálfarann

In [5]:
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("jonfd/gpt2-igc-is")

training_args = TrainingArguments(
    output_dir="./gpt2-textar", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=16, # number of training epochs
    per_device_train_batch_size=64, # batch size for training
    per_device_eval_batch_size=128,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=600,# number of warmup steps for learning rate scheduler
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Þjálfum

In [ ]:
trainer.train()

# Vistum

In [1]:
from transformers import AutoModel
model = AutoModel.from_pretrained("gpt2-textar")
trainer.push_to_hub("ofurtumi/gpt2textar_v0.1", private=True)

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: gpt2-textar is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [3]:
from transformers import pipeline

gen1 = pipeline("text-generation", model="ofurtumi/gpt2textar_v0.1", tokenizer="jonfd/gpt2-igc-is", pad_token_id=50256)
gen2 = pipeline("text-generation", model="ofurtumi/gpt2textar_v0.2", tokenizer="jonfd/gpt2-igc-is", pad_token_id=50256)
gen3 = pipeline("text-generation", model="ofurtumi/gpt2textar_v0.3", tokenizer="jonfd/gpt2-igc-is", pad_token_id=50256)

versions = {"v0.1": gen1, "v0.2": gen2, "v0.3": gen3}
# gen03 = pipeline("text-generation", model="ofurtumi/gpt2textar_v0.1", tokenizer="jonfd/gpt2-igc-is")

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def compare_versions(prompt):
    for name, version in versions.items():
        print(name)
        print(version(prompt, do_sample=True, temperature=0.9)[0]["generated_text"])
        print()

In [8]:
compare_versions("Kisuvæl")

v0.1
Kisuvæl
Komdu og dönsuðu fyrir mig
Komdu og dönsuðu fyrir mig
Komdu og dönsuðu fyrir mig
Komdu og dansaðu fyrir mig
Komdu og dansaðu fyrir mig
Kom

v0.2
Kisuvæl.
Lóan á morgnanna fer á stjá
og lóan á hádegi syngur sitt lag
Lóan á morgnana fer á stjá
og lóan á hádegi syngur sitt lag
Og þegar klukk

v0.3
Kisuvæl"Ég get ekki sungið,
ég get ekki sungið með þér
Það geta allir sungið
Það geta allir sungið
það geta allir grátið.  Ég hitti hana á bar í Berlín.
Hún brosti blítt til mín



In [9]:
compare_versions("Fólk sem hefur alltof gaman")

v0.1
Fólk sem hefur alltof gaman af því að hafa gaman,
svona venjulegt fólk,
ég myndi segja að það væri komið nóg.
En kannski var þetta allt saman vitleysa.  Ég er ekki einn af þeim sem nenna að hanga á klósettinu

v0.2
Fólk sem hefur alltof gaman
er löngu hætt að sýna á sér sparihliðarnar.
Ég er viss um að ég geti sagt ykkur,
ég er nokkuð viss um að ég geti sagt ykkur,
ég er nokkuð viss um að ég geti sagt

v0.3
Fólk sem hefur alltof gaman af því sem það sér
Á móti kemur samt að það er ekki þannig
Freistingarnar eru til að vaxa og vaxa en ekkert nema vonleysi
Það er ekki töff að vera grannur
Það er ekki töff að



In [10]:
compare_versions("Texti um barn sem týndi boltanum sínum")

v0.1
Texti um barn sem týndi boltanum sínum
og tók hann til sín
ég fann það strax í fyrstu hendi
Það var lítið um svefn í þessum bæ
og fátt sem var á sveimi
við fórum í labbitúr með honum
ég

v0.2
Texti um barn sem týndi boltanum sínum
En nú er hann kominn á leiðarenda
Og pabbi hans, er kominn heim
Og pabbi hans, er kominn heim
Er kominn heim  Skotta litla fer í bankann
og tekur út
Kass

v0.3
Texti um barn sem týndi boltanum sínum
Bamm bamm bamm bamm bamm belj
Bamm bamm bamm bamm
Bamm bamm bamm bamm belj
Bamm bamm bamm bamm



In [5]:
compare_versions("Jólahúfan er flott")

v0.1
Jólahúfan er flott og hlý
hún er góðhestur og gaman að fara í hana.
Nú er hún farin að draga mig
hittist við hana ömmu,
hún sagði nei, nei, við mig.


v0.2
Jólahúfan er flott
Við skulum fara í lautarferð
Því að þetta er hún.
Við notum hana í lautarferð
Og síðan förum við í lautarferð  Á dimmri nótt ég leggst upp í rúm til þín
Ég veit

v0.3
Jólahúfan er flott
úr því sem komið er
og úr því sem komið er
og af því ég veit
Að hér var það fyrir einum 15 árum
að búa í tjaldi
Og eiga með okkur kofa
í stormi




In [6]:
compare_versions("Bjór er góður")

v0.1
Bjór er góður.
Bjór er góður.
Bjór er góður.
Bjór er góður.
Við erum á þingi
og öll okkar vandamál eru leyst.
Og við látum ekki neitt stöðva okkur.
Bjór er góður

v0.2
Bjór er góður
mér finnst gott  Lestin er á enda,
sleppa gatan svo þröng.
Hopp og hæ og halló.
Ég gef henni sans,
alloft kenndur finnings
út á lífið

v0.3
Bjór er góður
þú veist að hann er góður
þú veist að hann er góður
þú veist að hann er svalur
þú veist að hann er feitur
þú veist að hann er æði
þú veist

